In [1]:
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import gensim

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data Loading

In [3]:
# Load Dataset from drive
fake_news_data = pd.read_csv('/content/drive/My Drive/MLSpring2020/TheMeanSquares-StockPrediction/Alternus-Vera TheMeanSquares/Iteration 1/Datasets/fake.csv', low_memory =False)
fake_news_data.head(2)

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias


# Preprocessing

In [0]:
fake_news_data = fake_news_data.dropna()

In [0]:
import re

def cleaning(raw_news):
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)

In [0]:
import time

t1 = time.time()
fake_news_data['clean_title'] = fake_news_data["title"].apply(cleaning) 
t2 = time.time()
print("\nTime to clean, tokenize and stem title in fake_news_data: \n", len(fake_news_data), "news:", (t2-t1)/60, "min")

t1 = time.time()
fake_news_data['clean_thread_title'] = fake_news_data["thread_title"].apply(cleaning) 
t2 = time.time()
print("\nTime to clean, tokenize and stem thread_title in fake_news_data: \n", len(fake_news_data), "news:", (t2-t1)/60, "min")

# Testing
# t1 = time.time()
# fake_news_data['clean_text'] = fake_news_data["text"].apply(cleaning) 
# t2 = time.time()
# print("\nTime to clean, tokenize and stem text in fake_news_data: \n", len(fake_news_data), "news:", (t2-t1)/60, "min")


Time to clean, tokenize and stem title in fake_news_data: 
 4702 news: 0.06761563618977864 min

Time to clean, tokenize and stem thread_title in fake_news_data: 
 4702 news: 0.03860367139180501 min


In [0]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/Shared drives/CMPE 257: Machine Learning/AlterusVera-Datasets/GoogleNews-vectors-negative300.bin.gz', binary=True)
words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [0]:
def spell_checker(text):
    all_words = re.findall(r'\w+', text.lower()) # split sentence to words
    spell_checked_text  = []
    for i in range(len(all_words)):
        spell_checked_text.append(correction(all_words[i]))
    return ' '.join(spell_checked_text)

In [0]:
print("Before: \n", fake_news_data['clean_title'][0] )
t1 = time.time()
fake_news_data['clean_title'] = fake_news_data['clean_title'].apply(spell_checker)
t2 = time.time()
print("\nTime to spell check the clean_title in fake_news_data: \n", len(fake_news_data), "news:", (t2-t1)/60, "min")

print("\nAfter: \n",fake_news_data['clean_title'][0] )
fake_news_data.head(2)

print("Before: \n", fake_news_data['clean_thread_title'][0] )
t1 = time.time()
fake_news_data['clean_thread_title'] = fake_news_data['clean_thread_title'].apply(spell_checker)
t2 = time.time()
print("\nTime to spell check the clean_thread_title in fake_news_data: \n", len(fake_news_data), "news:", (t2-t1)/60, "min")

print("\nAfter: \n",fake_news_data['clean_thread_title'][0] )
fake_news_data.head(2)

# Testing
# print("Before: \n", fake_news_data['clean_text'][0] )
# t1 = time.time()
# fake_news_data['clean_text'] = fake_news_data['clean_text'].apply(spell_checker)
# t2 = time.time()
# print("\nTime to spell check the clean_text in fake_news_data: \n", len(fake_news_data), "news:", (t2-t1)/60, "min")

# print("\nAfter: \n",fake_news_data['clean_text'][0] )
# fake_news_data.head()

Before: 
 muslim bust stole million gov benefit

Time to spell check the clean_title in fake_news_data: 
 4702 news: 2.643182162443797 min

After: 
 muslim bust stole million gov benefit
Before: 
 muslim bust stole million gov benefit

Time to spell check the clean_thread_title in fake_news_data: 
 4702 news: 2.6301859815915427 min

After: 
 muslim bust stole million gov benefit


,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type,clean_title,clean_thread_title
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,muslim bust stole million gov benefit,muslim bust stole million gov benefit
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,attorney general lotta lynch plead fifth,attorney general lotta lynch plead fifth


In [0]:
fake_news_data['type'].value_counts()

bs            4064
bias           271
conspiracy     163
hate            99
junksci         97
state            6
fake             2
Name: type, dtype: int64

## LDA

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from gensim.models import CoherenceModel
stemmer=PorterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [0]:
processed_docs = fake_news_data['clean_title'].map(preprocess)
processed_docs[:10]

0              [muslim, bust, steal, million, benefit]
1        [attorney, gener, lotta, lynch, plead, fifth]
2    [break, weiner, cooper, hillari, email, investig]
3    [drop, speech, father, daughter, kidnap, kill,...
4    [fantasi, trump, point, plan, reform, healthca...
5    [hillari, absolut, berserk, protest, ralli, vi...
6    [break, readi, arrest, weiner, case, hillari, ...
7    [whistleblow, tell, chill, stori, massiv, vote...
8    [break, clinton, clear, coordi, minut, trick, ...
9    [evil, hillari, support, yell, trump, burn, tr...
Name: clean_title, dtype: object

In [0]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 benefit
1 bust
2 million
3 muslim
4 steal
5 attorney
6 fifth
7 gener
8 lotta
9 lynch
10 plead


In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(36, 1), (66, 1)]

In [0]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 36 ("tell") appears 1 time.
Word 66 ("america") appears 1 time.


In [0]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5953727044797885),
 (1, 0.46794196630692375),
 (2, 0.3915172951786276),
 (3, 0.5227579425558675)]


In [0]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=k, id2word=dictionary, passes=10, alpha=a, eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, dictionary=dictionary, coherence='c_v', texts=fake_news_data['clean_title'])
    
    return coherence_model_lda.get_coherence()

In [0]:
'''
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(bow_corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(bow_corpus, num_of_docs*0.75), 
               bow_corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total = (len(corpus_sets)*len(topics_range)*len(alpha)*len(beta)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            for a in alpha:
              # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                                k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()
'''

"\nimport numpy as np\nimport tqdm\ngrid = {}\ngrid['Validation_Set'] = {}\n# Topics range\nmin_topics = 2\nmax_topics = 11\nstep_size = 1\ntopics_range = range(min_topics, max_topics, step_size)\n# Alpha parameter\nalpha = list(np.arange(0.01, 1, 0.3))\nalpha.append('symmetric')\nalpha.append('asymmetric')\n# Beta parameter\nbeta = list(np.arange(0.01, 1, 0.3))\nbeta.append('symmetric')\n# Validation sets\nnum_of_docs = len(bow_corpus)\ncorpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), \n               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), \n               gensim.utils.ClippedCorpus(bow_corpus, num_of_docs*0.75), \n               bow_corpus]\ncorpus_title = ['75% Corpus', '100% Corpus']\nmodel_results = {'Validation_Set': [],\n                 'Topics': [],\n                 'Alpha': [],\n                 'Beta': [],\n                 'Coherence': []\n                }\n# Can take a long time to run\nif 1 == 1:\n    pbar = tqdm.tqdm(total = (len(c

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.037*"peopl" + 0.035*"water" + 0.032*"kill" + 0.023*"wikileak" + 0.022*"face" + 0.021*"scientist" + 0.020*"child" + 0.019*"trump" + 0.017*"syrian" + 0.017*"cancer"
Topic: 1 
Words: 0.059*"obama" + 0.042*"hillari" + 0.041*"trump" + 0.027*"clinton" + 0.026*"peopl" + 0.024*"explain" + 0.024*"year" + 0.021*"evil" + 0.020*"happen" + 0.019*"reveal"
Topic: 2 
Words: 0.053*"world" + 0.040*"trump" + 0.040*"hillari" + 0.031*"clinton" + 0.031*"video" + 0.029*"email" + 0.024*"right" + 0.023*"putin" + 0.021*"human" + 0.015*"news"
Topic: 3 
Words: 0.069*"clinton" + 0.047*"stand" + 0.042*"rock" + 0.036*"thier" + 0.036*"messin" + 0.036*"hillari" + 0.029*"protest" + 0.019*"polic" + 0.019*"begin" + 0.019*"state"
Topic: 4 
Words: 0.042*"clinton" + 0.033*"trump" + 0.030*"syria" + 0.026*"vote" + 0.023*"hillari" + 0.022*"wikileak" + 0.021*"hou" + 0.018*"elect" + 0.018*"speak" + 0.017*"machin"
Topic: 5 
Words: 0.106*"trump" + 0.038*"govern" + 0.035*"russia" + 0.032*"america" + 0.031*"hillar

In [0]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.045*"hillari" + 0.020*"clinton" + 0.020*"work" + 0.018*"chart" + 0.017*"eastern" + 0.017*"outlook" + 0.015*"wikileak" + 0.014*"help" + 0.014*"stand" + 0.013*"german"
Topic: 1 Word: 0.034*"elect" + 0.024*"water" + 0.023*"democrat" + 0.018*"polic" + 0.018*"state" + 0.017*"break" + 0.017*"video" + 0.015*"kill" + 0.014*"trump" + 0.013*"happen"
Topic: 2 Word: 0.047*"trump" + 0.030*"medium" + 0.027*"right" + 0.023*"putin" + 0.019*"power" + 0.015*"clinton" + 0.014*"elect" + 0.013*"hillari" + 0.013*"human" + 0.013*"contain"
Topic: 3 Word: 0.040*"trump" + 0.035*"video" + 0.031*"clinton" + 0.021*"hillari" + 0.020*"obama" + 0.020*"report" + 0.017*"investig" + 0.016*"come" + 0.015*"leav" + 0.014*"polit"
Topic: 4 Word: 0.035*"america" + 0.024*"trump" + 0.020*"comment" + 0.017*"lose" + 0.015*"clinton" + 0.014*"surviv" + 0.014*"hillari" + 0.013*"iller" + 0.012*"fake" + 0.011*"time"
Topic: 5 Word: 0.038*"world" + 0.035*"russia" + 0.028*"time" + 0.020*"hillari" + 0.017*"woman" + 0.016*

In [0]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))



Score: 0.699950635433197	 
Topic: 0.106*"trump" + 0.038*"govern" + 0.035*"russia" + 0.032*"america" + 0.031*"hillari" + 0.023*"real" + 0.023*"medium" + 0.021*"vote" + 0.021*"elect" + 0.020*"famili"

Score: 0.03335343301296234	 
Topic: 0.057*"elect" + 0.033*"black" + 0.025*"hillari" + 0.025*"clinton" + 0.023*"human" + 0.023*"life" + 0.021*"trump" + 0.019*"video" + 0.015*"iran" + 0.014*"american"

Score: 0.03334243595600128	 
Topic: 0.037*"peopl" + 0.035*"water" + 0.032*"kill" + 0.023*"wikileak" + 0.022*"face" + 0.021*"scientist" + 0.020*"child" + 0.019*"trump" + 0.017*"syrian" + 0.017*"cancer"

Score: 0.03334137424826622	 
Topic: 0.079*"trump" + 0.039*"presid" + 0.031*"dinucci" + 0.031*"mansion" + 0.026*"elect" + 0.022*"million" + 0.021*"donald" + 0.019*"time" + 0.018*"work" + 0.017*"truth"

Score: 0.033341046422719955	 
Topic: 0.053*"world" + 0.040*"trump" + 0.040*"hillari" + 0.031*"clinton" + 0.031*"video" + 0.029*"email" + 0.024*"right" + 0.023*"putin" + 0.021*"human" + 0.015*"news"

In [0]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6999488472938538	 
Topic: 0.035*"america" + 0.024*"trump" + 0.020*"comment" + 0.017*"lose" + 0.015*"clinton" + 0.014*"surviv" + 0.014*"hillari" + 0.013*"iller" + 0.012*"fake" + 0.011*"time"

Score: 0.033352065831422806	 
Topic: 0.025*"para" + 0.021*"halloween" + 0.019*"elect" + 0.017*"black" + 0.017*"health" + 0.015*"life" + 0.015*"home" + 0.015*"trump" + 0.014*"hillari" + 0.013*"clinton"

Score: 0.03334273025393486	 
Topic: 0.047*"trump" + 0.030*"medium" + 0.027*"right" + 0.023*"putin" + 0.019*"power" + 0.015*"clinton" + 0.014*"elect" + 0.013*"hillari" + 0.013*"human" + 0.013*"contain"

Score: 0.03334052115678787	 
Topic: 0.031*"trump" + 0.028*"donald" + 0.026*"american" + 0.019*"truth" + 0.017*"know" + 0.016*"white" + 0.015*"reveal" + 0.015*"announc" + 0.014*"poll" + 0.013*"review"

Score: 0.03333751857280731	 
Topic: 0.033*"vote" + 0.029*"hillari" + 0.026*"trump" + 0.024*"elect" + 0.020*"govern" + 0.018*"clinton" + 0.018*"support" + 0.017*"thing" + 0.015*"danger" + 0.013*"

# Data Insights

Topic: 0 Word: 0.045*"hillari" + 0.020*"clinton" + 0.020*"work" + 0.018*"chart" + 0.017*"eastern" + 0.017*"outlook" + 0.015*"wikileak" + 0.014*"help" + 0.014*"stand" + 0.013*"german"

Topic: 1 Word: 0.034*"elect" + 0.024*"water" + 0.023*"democrat" + 0.018*"polic" + 0.018*"state" + 0.017*"break" + 0.017*"video" + 0.015*"kill" + 0.014*"trump" + 0.013*"happen"

Topic: 2 Word: 0.047*"trump" + 0.030*"medium" + 0.027*"right" + 0.023*"putin" + 0.019*"power" + 0.015*"clinton" + 0.014*"elect" + 0.013*"hillari" + 0.013*"human" + 0.013*"contain"

Topic: 3 Word: 0.040*"trump" + 0.035*"video" + 0.031*"clinton" + 0.021*"hillari" + 0.020*"obama" + 0.020*"report" + 0.017*"investig" + 0.016*"come" + 0.015*"leav" + 0.014*"polit"

Topic: 4 Word: 0.035*"america" + 0.024*"trump" + 0.020*"comment" + 0.017*"lose" + 0.015*"clinton" + 0.014*"surviv" + 0.014*"hillari" + 0.013*"iller" + 0.012*"fake" + 0.011*"time"

Topic: 5 Word: 0.038*"world" + 0.035*"russia" + 0.028*"time" + 0.020*"hillari" + 0.017*"woman" + 0.016*"trump" + 0.013*"attack" + 0.013*"stop" + 0.012*"look" + 0.012*"refug"

Topic: 6 Word: 0.035*"clinton" + 0.033*"hillari" + 0.021*"link" + 0.020*"presid" + 0.020*"trump" + 0.015*"email" + 0.015*"mogul" + 0.014*"cancer" + 0.014*"peopl" + 0.013*"speak"

Topic: 7 Word: 0.025*"para" + 0.021*"halloween" + 0.019*"elect" + 0.017*"black" + 0.017*"health" + 0.015*"life" + 0.015*"home" + 0.015*"trump" + 0.014*"hillari" + 0.013*"clinton"

Topic: 8 Word: 0.033*"vote" + 0.029*"hillari" + 0.026*"trump" + 0.024*"elect" + 0.020*"govern" + 0.018*"clinton" + 0.018*"support" + 0.017*"thing" + 0.015*"danger" + 0.013*"need"

Topic: 9 Word: 0.031*"trump" + 0.028*"donald" + 0.026*"american" + 0.019*"truth" + 0.017*"know" + 0.016*"white" + 0.015*"reveal" + 0.015*"announc" + 0.014*"poll" + 0.013*"review"

Once again, we ran this on 10 topics, and we noticed that the articles were political again.
From the topics we can see they talk about Hillary Clinton and the investigation into the leaks, as well as Donald Trump and the campaign.  

These once again show the bias, that could mislead the voters.
We can use LDA to determine that spelling seems to be a weak point for fake articles, as well as they happen to have a large bias.
This data can be used to implement a spell checker as well as a clickbait checker. This will give us some promising results.


# References
* https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
* https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
* https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0